In [18]:
from concurrent.futures import ThreadPoolExecutor
import json
import os
from openai import OpenAI
import logging
from time import perf_counter, sleep
import re
from threading import RLock
from dotenv import load_dotenv

load_dotenv()

log_format = '[ %(levelname)s] [%(asctime)s] [%(module)s] [%(lineno)s] [%(message)s]'
logging.basicConfig(level=logging.DEBUG, format=log_format)

file_handler = logging.FileHandler('logfile.log')
file_handler.setLevel(logging.DEBUG)

formatter = logging.Formatter(log_format)
file_handler.setFormatter(formatter)

log = logging.getLogger()
log.addHandler(file_handler)

lock = RLock()
converted_qno = []
failed_qno = []
reponse_data = []



In [19]:

SYSTEM_PROMPT = """
**You are a UPSC prelims question expert specializing in converting English MCQs to Hindi.**

**Given a question in English (statement) and a hint, convert it to Hindi (statement) suitable for the UPSC prelims exam, maintaining technical accuracy and UPSC context.**

**Input (JSON):**

* `statement`: Question statement with options in English.
* `hint`: Hint for the question in English.

**Output (JSON):**

* `statement`: Converted question statement with options in Hindi.
* `hint`: Hint for the question in Hindi.

**IMPORTANT INFORMATION**
- DONOT change the base statement format. The number of options should remain the same in the converted statement.
- DONOT forget to translate choices from end of each input `statement` to reponse `statement`.
"""

In [20]:
def clean_text(text, clean=True):
    if not clean:
        return re.sub(r'\s+', ' ', text.strip())
        # return text.strip()
    return re.sub(r'\s+', ' ', text.replace("\\n", " ").replace("", " ").strip())


In [21]:
def get_data():
    input_data = []
    input_data_dict = {}
    qno = []
    with open('data.json', 'r') as fp:
        json_data = json.load(fp)
    for key, item in json_data.items():
        statement = clean_text(item['statement'])
        hint = clean_text(item['hint'])
        qno = item["Qno"]
        temp = {'statement': statement, 'hint': hint, "Qno": qno}
        input_data.append(temp)
        input_data_dict[key] = temp
    # save_input_data(input_data_dict)
    return input_data


In [22]:
def convert_question(input_data, OPENAI_KEY):
    qno = input_data.pop("Qno")
    if qno in converted_qno:
        log.debug(f'{qno} already converted')
        return
    try:
        USER_PROMPT = str(input_data)
        data = OpenAI.generate_completion(SYSTEM_PROMPT, USER_PROMPT, OPENAI_KEY)
        if not data:
            raise Exception('Opena ai didnot respond')
        data["Qno"] = qno
        with lock:
            converted_qno.append(qno)
        reponse_data.append(data)
        # with lock:
            # save_data(reponse_data)

    except Exception as e:
        log.error(str(e))
        # with lock:
        failed_qno.append(qno)

In [23]:
def get_converted_questions():
    question_path = 'converted/questions_1.json'
    if os.path.exists(question_path):
        with open(question_path, 'r') as fp:
            file_data = json.load(fp)
        return [data['Qno'] for data in file_data]
    return []

In [24]:
def save_data():
    question_path = 'converted/questions_1.json'
    file_data = []
    try:
        if os.path.exists(question_path):
            with open(question_path, 'r') as fp:
                file_data = json.load(fp)

            with open(f'converted/questions_1_copy.json', 'w') as fp:
                json.dump(file_data, fp, ensure_ascii=False)

        file_data.extend(reponse_data)
        with open(question_path, 'w') as fp:
            json.dump(file_data, fp, ensure_ascii=False)
        return True

    except Exception as e:
        log.error(str(e))
        return None

In [25]:
input_data = get_data()
converted_qno.extend(get_converted_questions())
OPENAI_KEY_1 = os.getenv('OPENAI_KEY_1')

log.info(converted_qno)

[ INFO] [2024-04-29 15:26:53,532] [3737438831] [6] [[{'statement': 'संसद में राज्यों के प्रतिनिधित्व के प्रति निम्नलिखित कथनों पर विचार करें: 1. निर्वाचन क्षेत्रों का सीमांकन जनगणना अभ्यास के आधार पर किया जाता है ताकि सुनिश्चित हो कि प्रत्येक राज्य को संसद के दोनों सदनों में अपनी जनसंख्या के अनुपात में प्रतिनिधित्व मिले। 2. सीमांकन आयोग एक संवैधानिक निकाय है, जिसके आदेश की अधिसूचना किसी भी न्यायालय में चुनौती नहीं की जा सकती है। 3. राज्यों में क्षेत्रीय निर्वाचनी क्षेत्र, वर्तमान में, 2001 की जनगणना के आंकड़ों पर आधारित हैं, क्योंकि संविधान (87वां संशोधन) अधिनियम, 2003 ने 2001 की जनगणना के आंकड़ों पर आधारित सीमांकन अभ्यास को संभव बनाया। 4. जैसा कि आज है, भारतीय संविधान किसी भी सीमांकन अभ्यास को 2031 तक नहीं अनुमति देता है। उपर्युक्त कथनों में से कौन सही नहीं हैं?', 'hint': 'सीमांकन क्षेत्र राज्य सभा में प्रतिनिधित्व के लिए लागू नहीं होते। हालांकि यह सही है कि सीमांकन आयोग के आदेश, एक बार अधिसूचित किए जाने पर, किसी भी न्यायालय में चुनौती नहीं की जा सकती है, सीमांकन आयोग एक संवैधानिक निक

In [22]:
for data in input_data:
    qno = data["Qno"]
    log.info(f'Converting question {qno}')
    try:
        convert_question(data, OPENAI_KEY=OPENAI_KEY_1)
        save_data()
        log.info(f'Converted question {qno}')
    except Exception as e:
        log.error(str(e))


[ INFO] [2024-04-29 15:21:31,487] [3818965748] [3] [Converting question 0]
[ DEBUG] [2024-04-29 15:21:31,488] [2206373033] [4] [0 already converted]
[ INFO] [2024-04-29 15:21:31,491] [3818965748] [7] [Converted question 0]
[ INFO] [2024-04-29 15:21:31,492] [3818965748] [3] [Converting question 1]
[ DEBUG] [2024-04-29 15:21:31,493] [2206373033] [4] [1 already converted]
[ INFO] [2024-04-29 15:21:31,494] [3818965748] [7] [Converted question 1]
[ INFO] [2024-04-29 15:21:31,495] [3818965748] [3] [Converting question 10]
[ DEBUG] [2024-04-29 15:21:31,495] [2206373033] [4] [10 already converted]
[ INFO] [2024-04-29 15:21:31,497] [3818965748] [7] [Converted question 10]
[ INFO] [2024-04-29 15:21:31,498] [3818965748] [3] [Converting question 100]
[ DEBUG] [2024-04-29 15:21:31,502] [2206373033] [4] [100 already converted]
[ INFO] [2024-04-29 15:21:31,503] [3818965748] [7] [Converted question 100]
[ INFO] [2024-04-29 15:21:31,506] [3818965748] [3] [Converting question 1000]
[ DEBUG] [2024-04-29 

KeyboardInterrupt: 

In [9]:
reponse_data

[{'statement': 'संसद में राज्यों के प्रतिनिधित्व के प्रति निम्नलिखित कथनों पर विचार करें: 1. निर्वाचन क्षेत्रों का सीमांकन जनगणना अभ्यास के आधार पर किया जाता है ताकि सुनिश्चित हो कि प्रत्येक राज्य को संसद के दोनों सदनों में अपनी जनसंख्या के अनुपात में प्रतिनिधित्व मिले। 2. सीमांकन आयोग एक संवैधानिक निकाय है, जिसके आदेश की अधिसूचना किसी भी न्यायालय में चुनौती नहीं की जा सकती है। 3. राज्यों में क्षेत्रीय निर्वाचनी क्षेत्र, वर्तमान में, 2001 की जनगणना के आंकड़ों पर आधारित हैं, क्योंकि संविधान (87वां संशोधन) अधिनियम, 2003 ने 2001 की जनगणना के आंकड़ों पर आधारित सीमांकन अभ्यास को संभव बनाया। 4. जैसा कि आज है, भारतीय संविधान किसी भी सीमांकन अभ्यास को 2031 तक नहीं अनुमति देता है। उपर्युक्त कथनों में से कौन सही नहीं हैं?',
  'hint': 'सीमांकन क्षेत्र राज्य सभा में प्रतिनिधित्व के लिए लागू नहीं होते। हालांकि यह सही है कि सीमांकन आयोग के आदेश, एक बार अधिसूचित किए जाने पर, किसी भी न्यायालय में चुनौती नहीं की जा सकती है, सीमांकन आयोग एक संवैधानिक निकाय नहीं है बल्कि एक कानूनी निकाय है। संविधान ने रा

In [14]:
converted_qno

[]